In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import pickle as pickle
import os
import phenopy
import graphviz as gv
from pyvis.network import Network
import pyvis as vis
import random
import ssmpy
import numpy as np
from sklearn import metrics

import os
from phenopy.build_hpo import generate_annotated_hpo_network
from phenopy.score import Scorer

In [ ]:
#Extracting data 
data = pd.read_pickle(file) # Unfortunately the dataset is not publicly available
hpo_all_p = data.hpo_all_with_parents
hpo_all_n = data.hpo_all_name
hpo_all = data.hpo_all
hpo_name_par = data.hpo_all_name_with_parents
graphs = data.graphs

In [ ]:
# Removing non-phenotype nodes
to_remove_clinical = []
patients_with_clinicalmod = []
idx = 0
for g in graphs:
    if g.has_node('Clinical modifier'):
        ancestors = nx.ancestors(g,'Clinical modifier')
        ancestors.add('Clinical modifier')
        to_remove_clinical.append(ancestors)
        patients_with_clinicalmod.append(idx)
    idx+=1    
    
remove_279 = []
for synonym in to_remove_clinical[0]:
    if synonym in hpo_all_n[279]:
        remove_279.append(hpo_all_n[279].index(synonym))
    
remove_366 = []
for synonym in to_remove_clinical[1]:
    if synonym in hpo_all_n[366]:
        remove_366.append(hpo_all_n[366].index(synonym))
    
remove_454 = []
for synonym in to_remove_clinical[2]:
    if synonym in hpo_all_n[454]:
        remove_454.append(hpo_all_n[454].index(synonym))
    
remove_279.sort(reverse = True)
remove_366.sort(reverse = True)
remove_454.sort(reverse = True)


for ix in remove_279:
    hpo_all[279].pop(ix)
    
    
for ix in remove_366:
    hpo_all[366].pop(ix)
    
for ix in remove_454:
    hpo_all[454].pop(ix)

In [ ]:
# data directory
phenopy_data_directory = ""

# files used in building the annotated HPO network
obo_file = os.path.join(phenopy_data_directory, 'hp.obo')
disease_to_phenotype_file = os.path.join(phenopy_data_directory, 'phenotype.hpoa')

# if you have a custom ages_distribution_file, you can set it here.


hpo_network, alt2prim, disease_records = \
    generate_annotated_hpo_network(obo_file,
                                   disease_to_phenotype_file)

In [ ]:
# Removing alt_id's
for i in range(0,len(hpo_all)-1):    
    hpo_all[i] = [alt2prim[term] if term in alt2prim else term for term in hpo_all[i]]

In [ ]:
# Making a list with all patients that have kidney abnormalities
patients_kidney_abnormalities = []
patient_indexes_kidney = []
idx = 0
for patient in hpo_all_p:
    if "HP:0000077" in patient:
        patients_kidney_abnormalities.append(patient)
        patient_indexes_kidney.append(idx)
    idx+=1   

In [ ]:
# Making a list with all patients that have heart abnormalities
patients_heart_abnormalities = []
patient_indexes_heart = []
idx = 0
for patient in hpo_all_p:
    if "HP:0001627" in patient:
        patients_heart_abnormalities.append(patient)
        patient_indexes_heart.append(idx)
    idx+=1

In [ ]:
# Plot of ratio between heart patients and non heart patients
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
Classes = ['Patients with heart abnormality', 'All patients']
Patients = [len(patients_heart_abnormalities),(len(hpo_all_p)-len(patients_heart_abnormalities))]
ax.bar(Classes,Patients)
plt.title('Ratio between patients with heart abnormality and all patients')
plt.show()

# Plot of ratio between kidney patients and non kidney patients
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
Classes = ['Patients with kidney abnormality', 'All patients']
Patients = [len(patients_kidney_abnormalities),(len(hpo_all_p)-len(patients_kidney_abnormalities))]
ax.bar(Classes,Patients)
plt.title('Ratio between patients with a kidney abnormality and all patients')
plt.show()

In [ ]:
#Function that removes all kidney abnormalities from a patient
def remove_kidney_symptoms(index):
    patient_kidney = hpo_all[index]
    to_remove_kidney = []
    idx = 0
    if graphs[index].has_node('Abnormality of the kidney'):
        ancestors = nx.ancestors(graphs[index],'Abnormality of the kidney')
        ancestors.add('Abnormality of the kidney')
        to_remove_kidney.append(ancestors)
        
    remove_kidney_indexes = []
    for synonym in to_remove_kidney[0]:
        if synonym in hpo_all_n[index]:
            remove_kidney_indexes.append(hpo_all_n[index].index(synonym))

    remove_kidney_indexes.sort(reverse=True)


    for ix in remove_kidney_indexes:
        hpo_all[index].pop(ix)

In [ ]:
#Function that removes all heart abnormalities from a patient
def remove_heart_symptoms(index):
    patient_heart = hpo_all[index]
    to_remove_heart = []
    idx = 0
    if graphs[index].has_node('Abnormal heart morphology'):
        ancestors = nx.ancestors(graphs[index],'Abnormal heart morphology')
        ancestors.add('Abnormal heart morphology')
        to_remove_heart.append(ancestors)
        
    remove_heart_indexes = []
    for synonym in to_remove_heart[0]:
        if synonym in hpo_all_n[index]:
            remove_heart_indexes.append(hpo_all_n[index].index(synonym))

    remove_heart_indexes.sort(reverse=True)


    for ix in remove_heart_indexes:
        hpo_all[index].pop(ix)

In [ ]:
# Function that returns largest number of desired items from list sorted.
def find_largest(lst,num):
    return sorted( [(x,i) for (i,x) in enumerate(lst)], reverse=True )[:num] 

In [ ]:
#Function that returns Maximum Common subgraph of two graphs
def getMCS(gr1, gr2):
    overlapping_graph=nx.Graph()

    for nd1,nd2 in gr2.edges():
        if gr1.has_edge(nd1, nd2):
            overlapping_graph.add_edge(nd1, nd2)
            
    elements = nx.connected_components(overlapping_graph)
    largest_element = max(elements, key=len)
    return nx.induced_subgraph(overlapping_graph, largest_element)

In [ ]:
#Function that scales from 0 to 1
def scale(l,x):
    return (x-min(l))/(max(l)-min(l))

In [ ]:
#Calculating all three similarity matrices for kidney abnormalities

# Calculating Kidney similarity matrix with Resnik similiarity
scorer = Scorer(hpo_network,scoring_method="Resnik")
sim_matrix_resnik_nier = []
idx = 0
for patient in hpo_all:
    scores = []
    if idx in patient_indexes_kidney:
        remove_kidney_symptoms(idx)
    for p in hpo_all:
        scores.append(scorer.score_term_sets_basic(patient,p))
    sim_matrix_resnik_nier.append(scores)
    idx+=1
%store sim_matrix_resnik_nier

# Calculating Kidney similarity matrix with Lin similiarity
scorer_lin = Scorer(hpo_network,scoring_method="Lin")

sim_matrix_lin_nier= []
idx = 0
for patient in hpo_all:
    scores = []
    if idx in patient_indexes_kidney:
        remove_kidney_symptoms(idx)
    for p in hpo_all:
        scores.append(scorer_lin.score_term_sets_basic(patient,p))
    sim_matrix_lin_nier.append(scores)
    idx+=1
%store sim_matrix_lin_nier

# Calculating kidney similarity matrix with MCS

sim_matrix_mcs_nier= []
idx = 0
for patient in graphs:
    scores = []
    size = patient.number_of_nodes() + patient.number_of_edges()
    if idx in patient_indexes_kidney:
        remove_kidney_symptoms_graph(idx)
    for p in graphs:        
        mcs = getMCS(patient,p)
        score = (mcs.number_of_nodes() + mcs.number_of_edges()) / size # correcting for graph size
        scores.append(score)
    sim_matrix_mcs_nier.append(scores)
    idx+=1
%store sim_matrix_mcs_nier

In [ ]:
#Calculating similarity matrices for heart abnormalities

# Calculating heart similarity matrix with Resnik similiarity
scorer = Scorer(hpo_network,scoring_method="Resnik")

sim_matrix_resnik_heart = []
idx = 0
for patient in hpo_all:
    scores = []
    if idx in patient_indexes_heart:
        remove_heart_symptoms(idx)
    for p in hpo_all:
        scores.append(scorer.score_term_sets_basic(patient,p))
    sim_matrix_resnik_heart.append(scores)
    idx+=1
%store sim_matrix_resnik_heart


# Calculating heart similarity matrix with Lin similiarity
scorer_lin = Scorer(hpo_network,scoring_method="Lin")

sim_matrix_lin_heart= []
idx = 0
for patient in hpo_all:
    scores = []
    if idx in patient_indexes_heart:
        remove_heart_symptoms(idx)
    for p in hpo_all:
        scores.append(scorer_lin.score_term_sets_basic(patient,p))
    sim_matrix_lin_heart.append(scores)
    idx+=1
%store sim_matrix_lin


# Calculating heart similarity matrix with MCS
sim_matrix_mcs_heart= []
idx = 0
for patient in graphs:
    scores = []
    size = patient.number_of_nodes() + patient.number_of_edges()
    if idx in patient_indexes_heart:
        remove_heart_symptoms_graph(idx)
    for p in graphs:        
        mcs = getMCS(patient,p)
        score = (mcs.number_of_nodes() + mcs.number_of_edges()) / size
        scores.append(score)
    sim_matrix_mcs_heart.append(scores)
    idx+=1
%store sim_matrix_mcs_heart

In [ ]:
# Scaling kidney similarity matrices for the combination

scaled_resnik_nier = []
for lis in sim_matrix_resnik_nier:
    scaled = [scale(lis,i) for i in lis]
    scaled_resnik_nier.append(scaled)
    
scaled_lin_nier = []
for lis in sim_matrix_lin_nier:
    scaled = [scale(lis,i) for i in lis]
    scaled_lin_nier.append(scaled)
    
#Combining Kidney similarity matrices with each other to test combined methods


combresnikmcs_nier = [] #Resnik + MCS
for res,mcs in zip(scaled_resnik_nier,sim_matrix_mcs_nier):
    comb = [((i+j)/2) for i,j in zip(res,mcs)]
    combresnikmcs_nier.append(comb)
    
    
combresniklin_nier = [] #Resnik + Lin
for res,lin in zip(scaled_resnik_nier,scaled_lin_nier):
    comb = [((i+j)/2) for i,j in zip(res,lin)]
    combresniklin_nier.append(comb)
    
    
comblinmcs_nier = [] #Lin + MCS
for lin,mcs in zip(scaled_lin_nier,sim_matrix_mcs_nier):
    comb = [((i+j)/2) for i,j in zip(lin,mcs)]
    comblinmcs_nier.append(comb)

In [ ]:
# Scaling Heart similarity matrices for the combination

scaled_resnik_heart = []
for lis in sim_matrix_resnik_heart:
    scaled = [scale(lis,i) for i in lis]
    scaled_resnik_heart.append(scaled)
    
scaled_lin_heart = []
for lis in sim_matrix_lin_heart:
    scaled = [scale(lis,i) for i in lis]
    scaled_lin_heart.append(scaled)
    
#Combining Heart similarity matrices with each other to test combined methods


combresnikmcs_heart = [] #Resnik + MCS
for res,mcs in zip(scaled_resnik_heart,sim_matrix_mcs_heart):
    comb = [((i+j)/2) for i,j in zip(res,mcs)]
    combresnikmcs_heart.append(comb)
    
    
combresniklin_heart = [] #Resnik + Lin
for res,lin in zip(scaled_resnik_heart,scaled_lin_heart):
    comb = [((i+j)/2) for i,j in zip(res,lin)]
    combresniklin_heart.append(comb)
    
    
comblinmcs_heart = [] #Lin + MCS
for lin,mcs in zip(scaled_lin_heart,sim_matrix_mcs_heart):
    comb = [((i+j)/2) for i,j in zip(lin,mcs)]
    comblinmcs_heart.append(comb)

In [ ]:
#Calculating AUC for similarity matrix with variable nr of top most similar patients
def calc_auc_score(nr,sim_matrix,patient_indexes):

    idx = 0
    number_patients_consider = nr
    y_labels = np.array([1 if i in patient_indexes else 0 for i in range(0,len(hpo_all)) ])
    probabilities = []


    for patient in hpo_all:
        scores = sim_matrix[idx]    
        top = find_largest(scores,number_patients_consider) #Top most similar patients
        to_consider = []
        for i in range(1,number_patients_consider):
            to_consider.append(top[i][1])

        nr_abnormalities = 0 # Amount of abnormalities in top

        for ind in to_consider:
            if ind in patient_indexes: 
                nr_abnormalities+=1

        l = nr_abnormalities / nr # score

        probabilities.append(l)


        idx+=1
    
    auc = metrics.roc_auc_score(y_labels, probabilities) #calculating AUC
    
    return auc

In [ ]:
#Function that calculates AUC scores for certain matrix
def calc_scores(matrix,patient_indexes):
    auc_scores = []
    numbers = [1,5,10,25,50,100]
    for n in numbers:
        auc_scores.append(calc_auc_score(n,matrix,patient_indexes)
    return auc_scores

In [ ]:
# Calculating AUC scores for variable top Resnik with Kidney
auc_scores_res_nier = calc_scores(sim_matrix_resnik_nier,patient_indexes_kidney)
# Calculating AUC scores for variable top mcs with Kidney
auc_scores_mcs_nier = calc_scores(sim_matrix_mcs_nier,patient_indexes_kidney)
# Calculating AUC scores for variable top Lin with Kidney
auc_scores_mcs_nier = calc_scores(sim_matrix_lin_nier,patient_indexes_kidney)
# Calculating AUC scores for variable top resnik + MCS with Kidney
auc_scores_resmcs = calc_scores(combresnikmcs_nier,patient_indexes_kidney)
# Calculating AUC scores for variable top Resnik + Lin with Kidney
auc_scores_reslin = calc_scores(combresniklin_nier,patient_indexes_kidney)
# Calculating AUC scores for variable top Lin + MCS with Kidney
auc_scores_linmcs = calc_scores(comblinmcs_nier,patient_indexes_kidney)

In [ ]:
# Calculating AUC scores for variable top Resnik with heart
auc_scores_res_heart = calc_scores(sim_matrix_resnik_heart,patient_indexes_heart)
# Calculating AUC scores for variable top mcs with heart
auc_scores_mcs_heart = calc_scores(sim_matrix_mcs_heart,patient_indexes_heart)
# Calculating AUC scores for variable top Lin with heart
auc_scores_mcs_heart = calc_scores(sim_matrix_lin_heart,patient_indexes_heart)
# Calculating AUC scores for variable top resnik + MCS with heart
auc_scores_resmcs_heart = calc_scores(combresnikmcs_heart,patient_indexes_heart)
# Calculating AUC scores for variable top Resnik + Lin with heart
auc_scores_reslin_heart = calc_scores(combresniklin_heart,patient_indexes_heart)
# Calculating AUC scores for variable top Lin + MCS with heart
auc_scores_linmcs_heart = calc_scores(comblinmcs_heart,patient_indexes_heart)

In [ ]:
#Plotting them
import matplotlib.pyplot as plt
fig, (ax1,ax2,ax3) = plt.subplots(3)
fig.suptitle('Auc scores of different classification rules across different graph similarity methods')
Classes = [1,5,10,25,50,100]
fig.set_figheight(25)
fig.set_figwidth(25)
#ax3.set_xticklabels([1,5,10,25,50,100])
ax1.bar(Classes,auc_scores_mcs)
ax1.set_title("auc scores with most common subgraph measure:")
ax2.bar(Classes,auc_scores_res)
ax2.set_title("auc scores with resnik measure: ")
ax3.bar(Classes,auc_scores_lin)
ax3.set_title("auc scores with Lin measure: ")
plt.show()

In [ ]:
# Can be used with a threshold to classify patients
def classifier_mcs(threshold,nr,matrix):
    idx = 0
    y_pred = []
    l = calc_likelihood(nr,matrix)
    for patient in hpo_all:
        if l[idx]>threshold:
            y_pred.append(1)
        else:
            y_pred.append(0)
        idx+=1
    return y_pred

In [ ]:
def calc_likelihood(nr,sim_matrix,patient_indexes):
    idx = 0
    number_patients_consider = nr
    y_labels = np.array([1 if i in patient_indexes else 0 for i in range(0,len(hpo_all)) ])
    likelihood = []


    for patient in hpo_all:
        scores = sim_matrix[idx]
        
        top = find_largest(scores,number_patients_consider)
        to_consider = []
        for i in range(1,number_patients_consider):
            to_consider.append(top[i][1])

        nr_kidney = 0
        for ind in to_consider:
            if ind in patient_indexes: 
                nr_kidney+=1
                

        l = nr_kidney / nr

        likelihood.append(l)


        idx+=1
    
    return likelihood

In [ ]:
#Plotting ROC curves for kidney classifiers

import matplotlib.pyplot as plt

fig, (ax1, ax2, ax3) = plt.subplots(1, 3,figsize=(10, 3))


fig.tight_layout(pad=4.0)


likelihood_res = calc_likelihood(50,sim_matrix_resnik_nier,patient_indexes_kidney)

y_labels = np.array([1 if i in patient_indexes_kidney else 0 for i in range(0,len(hpo_all))])
fpr, tpr, thresholds = metrics.roc_curve(y_labels, likelihood_res)
roc_auc = metrics.auc(fpr, tpr)

fig.suptitle('Receiver Operating Characteristic for all three methods with Kidney abnormality',fontname="Calibri",fontsize=14)
ax1.set_title('ROC Resnik')
ax1.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
ax1.legend(loc = 'lower right')
ax1.plot([0, 1], [0, 1],'r--')
ax1.set_xlim([0, 1])
ax1.set_xlim([0, 1])
ax1.set(xlabel='False Positive Rate', ylabel='True Positive Rate')



likelihood_lin = calc_likelihood(25,sim_matrix_lin_nier,patient_indexes_kidney)
ax2.set_title('ROC Lin')
y_labels = np.array([1 if i in patient_indexes_kidney else 0 for i in range(0,len(hpo_all))])
fpr, tpr, thresholds = metrics.roc_curve(y_labels, likelihood_lin)
roc_auc = metrics.auc(fpr, tpr)


ax2.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
ax2.legend(loc = 'lower right')
ax2.plot([0, 1], [0, 1],'r--')
ax2.set_xlim([0, 1])
ax2.set_xlim([0, 1])
ax2.set(xlabel='False Positive Rate', ylabel='True Positive Rate')


likelihood_mcs = calc_likelihood(100,sim_matrix_mcs_nier,patient_indexes_kidney)
ax3.set_title('ROC MCS')
y_labels = np.array([1 if i in patient_indexes_kidney else 0 for i in range(0,len(hpo_all))])
fpr, tpr, thresholds = metrics.roc_curve(y_labels, likelihood_mcs)
roc_auc = metrics.auc(fpr, tpr)


ax3.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
ax3.legend(loc = 'lower right')
ax3.plot([0, 1], [0, 1],'r--')
ax3.set_xlim([0, 1])
ax3.set_xlim([0, 1])
ax3.set(xlabel='False Positive Rate', ylabel='True Positive Rate')


In [ ]:
#Plotting ROC curves for Heart classifiers
import matplotlib.pyplot as plt

fig, (ax1, ax2, ax3) = plt.subplots(1, 3,figsize=(10, 3))


fig.tight_layout(pad=4.0)


likelihood_res = calc_likelihood(100,sim_matrix_resnik_heart,patient_indexes_heart)

y_labels = np.array([1 if i in patient_indexes_heart else 0 for i in range(0,len(hpo_all))])
fpr, tpr, thresholds = metrics.roc_curve(y_labels, likelihood_res)
roc_auc = metrics.auc(fpr, tpr)

fig.suptitle('Receiver Operating Characteristic for all three methods with Heart abnormality',fontname="Calibri",fontsize=14)
ax1.set_title('ROC Resnik')
ax1.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
ax1.legend(loc = 'lower right')
ax1.plot([0, 1], [0, 1],'r--')
ax1.set_xlim([0, 1])
ax1.set_xlim([0, 1])
ax1.set(xlabel='False Positive Rate', ylabel='True Positive Rate')



likelihood_lin = calc_likelihood(50,sim_matrix_lin_heart,patient_indexes_heart)
ax2.set_title('ROC Lin')
y_labels = np.array([1 if i in patient_indexes_heart else 0 for i in range(0,len(hpo_all))])
fpr, tpr, thresholds = metrics.roc_curve(y_labels, likelihood_lin)
roc_auc = metrics.auc(fpr, tpr)


ax2.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
ax2.legend(loc = 'lower right')
ax2.plot([0, 1], [0, 1],'r--')
ax2.set_xlim([0, 1])
ax2.set_xlim([0, 1])
ax2.set(xlabel='False Positive Rate', ylabel='True Positive Rate')


likelihood_mcs = calc_likelihood(100,sim_matrix_mcs_heart,patient_indexes_heart)
ax3.set_title('ROC MCS')
y_labels = np.array([1 if i in patient_indexes_heart else 0 for i in range(0,len(hpo_all))])
fpr, tpr, thresholds = metrics.roc_curve(y_labels, likelihood_mcs)
roc_auc = metrics.auc(fpr, tpr)


ax3.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
ax3.legend(loc = 'lower right')
ax3.plot([0, 1], [0, 1],'r--')
ax3.set_xlim([0, 1])
ax3.set_xlim([0, 1])
ax3.set(xlabel='False Positive Rate', ylabel='True Positive Rate')
